# MidJourney


In [12]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import  Dense, Reshape, Dropout, Flatten,Input
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
import numpy as np
import matplotlib.pyplot as plt

generator = Sequential() 
generator.add(Dense(128 * 7 * 7, input_dim=100))
generator.add(Activation(LeakyReLU(0.2)))
generator.add(Reshape((7,7,128))) 
generator.add(UpSampling2D()) 
generator.add(Conv2D(128, kernel_size=3, padding='same'))  
generator.add(BatchNormalization())  
generator.add(Activation(LeakyReLU(0.2))) 
generator.add(UpSampling2D())  
generator.add(Conv2D(64, kernel_size=3, padding='same')) 
generator.add(BatchNormalization()) 
generator.add(Activation(LeakyReLU(0.2)))  
generator.add(Conv2D(1, kernel_size=3, padding='same', activation='tanh'))

discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28,28,1), padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.trainable=False

ginput = Input(shape=(100,))
dis_output = discriminator(generator(ginput))
gan = Model(ginput, dis_output)
gan.compile(loss = 'binary_crossentropy', optimizer = 'adam')
gan.summary()

# 신경망 실행시키는 함수
def gan_train(epoch, batch_size, saving_interval) :

  #MNIST
  (X_train,_), (_, _) = mnist.load_data()
  X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
  X_train = (X_train - 127.5)/127.5

  true = np.ones((batch_size,1))
  fake = np.zeros((batch_size, 1))

  for i in range(epoch) :
    # 실제 데이터를 판별자에 입력
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]
    d_loss_real = discriminator.train_on_batch(imgs, true)

    # 가상 이미지를 판별자에 입력
    noise = np.random.normal(0, 1, (batch_size, 100))
    gen_imgs = generator.predict(noise)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)

    # 판별자와 생성자의 오차를 계산
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    g_loss = gan.train_on_batch(noise,true)

    print('epoch:%d' % i, 'd_loss:%.4f'% d_loss, 'g_loss%.4f'%g_loss)

    if i % saving_interval ==0:
      # r,c = 5, 5
      noise = np.random.normal(0,1,(25,100))
      gen_imgs = generator.predict(noise)
      # rescale images 0-1
      gen_imgs = 0.5 * gen_imgs + 0.5

      fig, axs = plt.subplots(5,5)
      count = 0
      for j in range(5):
        for k in range(5):
          axs[j,k].imshow(gen_imgs[count, :, :, 0], cmap='gray')
          axs[j,k].axis('off')
          count +=1
          fig.savefig('/content/drive/MyDrive/Colab Notebooks/gan_images/gan_mnist_%d.png'%i)
      


Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential_16 (Sequential)  (None, 28, 28, 1)         856193    
                                                                 
 sequential_17 (Sequential)  (None, 1)                 212865    
                                                                 
Total params: 1,069,058
Trainable params: 855,809
Non-trainable params: 213,249
_________________________________________________________________


In [ ]:
gan_train(4001,32,200)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1/1 [==============================] - 0s 156ms/step
epoch:1508 d_loss:0.4702 g_loss1.4407
1/1 [==============================] - 0s 261ms/step
epoch:1509 d_loss:0.5838 g_loss1.3232
1/1 [==============================] - 0s 245ms/step
epoch:1510 d_loss:0.4019 g_loss1.5968
1/1 [==============================] - 0s 217ms/step
epoch:1511 d_loss:0.5760 g_loss1.7747
1/1 [==============================] - 0s 143ms/step
epoch:1512 d_loss:0.4956 g_loss1.6227
1/1 [==============================] - 0s 148ms/step
epoch:1513 d_loss:0.6060 g_loss1.5803
1/1 [==============================] - 0s 143ms/step
epoch:1514 d_loss:0.3803 g_loss1.6186
1/1 [==============================] - 0s 147ms/step
epoch:1515 d_loss:0.5721 g_loss1.4887
1/1 [==============================] - 0s 140ms/step
epoch:1516 d_loss:0.4185 g_loss1.4622
1/1 [==============================] - 0s 140ms/step
epoch:1517 d_loss:0.4952 g_loss1.5045
1/1 [==============================] - 0s 140ms/step
e

<ipython-input-12-6d02f106729d>:75: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axs = plt.subplots(5,5)
